In [8]:
import pymysql
import pprint
pp = pprint.PrettyPrinter(indent = 4)

In [9]:
conn = pymysql.connect(host='localhost', user='admin',
                            passwd="password", db='joe_rogan')
cur = conn.cursor()
cur.execute(
    "SElECT id, episode_id, text FROM `transcript_time_split` "
    "WHERE episode_id=269 "
    "ORDER BY id ASC "
    "LIMIT 10000")
data_tuple = cur.fetchall()
cur.close()
conn.close()

In [10]:
data = []
for i in data_tuple:
    data.append(list(i))

In [11]:
for i in range(len(data)):
    data[i][2] = " ".join(data[i][2].split())

In [12]:
from transformers import pipeline
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
print(torch.cuda.is_available())
"""
    Google: google/pegasus-xsum 
    Google lg: google/pegasus-large 
    Facebook: facebook/bart-large-cnn
    SShleifer: sshleifer/distilbart-cnn-12-6
    google/pegasus-reddit_tifu

    Activate GPU by after model by writing: , device=0
"""
summarizer = pipeline("summarization", "google/pegasus-xsum", device=0)

True


In [13]:
output_summaries = []

for i in data:
    bitesizedtext = ""
    textlist = []
    tempEpisodeID = i[1]
    tempID = i[0]

    for x in i[2].split('.'):
        if (len(bitesizedtext.split())+len(x.split())) > 350:
            textlist.append(bitesizedtext)
            bitesizedtext = ""
            bitesizedtext = bitesizedtext + " " + x
        else:
            bitesizedtext = bitesizedtext + " " + x
            
            
    print(f"Episode: {tempEpisodeID} 10 minutes ID: {tempID}")

    summary = ""
    for y in textlist:
        text = summarizer(y, max_length=75, min_length=5, do_sample=False)[0]['summary_text']
        summary += text
    

    output_summaries.append([tempEpisodeID, tempID, summary])
    

Episode: 269 10 minutes ID: 2271
Episode: 269 10 minutes ID: 2272
Episode: 269 10 minutes ID: 2273
Episode: 269 10 minutes ID: 2274
Episode: 269 10 minutes ID: 2275
Episode: 269 10 minutes ID: 2276
Episode: 269 10 minutes ID: 2277
Episode: 269 10 minutes ID: 2278
Episode: 269 10 minutes ID: 2279
Episode: 269 10 minutes ID: 2280
Episode: 269 10 minutes ID: 2281
Episode: 269 10 minutes ID: 2282
Episode: 269 10 minutes ID: 2283


In [14]:
with open ('transformer_summaries.txt','a') as f:
    
    for i in range(len(output_summaries)):
        f.write(f"""
        EpisodeID: {output_summaries[i][0]}
        10 minute ID: {output_summaries[i][1]}
        Summary: 
        {output_summaries[i][2]}
        """
        )
        f.write(f"""
        Length of original: {len(data[i][2])}, Length of summary: {len(output_summaries[i][2])}
        
        """
        )
